# Code to compute $\sigma(f_{NL})$

We only return $\sigma(f_{NL})$, and not $\sigma(b_g)$.

## First for one tracer

In [8]:
import numpy as np
import scipy as sp
#import pyccl as ccl
import math
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
from functools import partial


from scipy.integrate import quad, dblquad
import pyccl as ccl

cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.677, A_s=2.1e-9, n_s=0.968,transfer_function='boltzmann_camb')
Omega_m=cosmo['Omega_m']
Omega_b=cosmo['Omega_b']
Omega_c=cosmo['Omega_c']
Omega_l=1-Omega_m
#sky=14000#1400#7000

def Vsurvey(z,dz,sky):    
    Omega     = sky*(math.pi/180)**2 # get rid of unit
    d2        = ccl.comoving_radial_distance(cosmo,1/(1+z))
    d3        = ccl.comoving_radial_distance(cosmo,1/(1+z+dz))
    return Omega/3 * (d3**3 - d2**3)*h**3
   
    
deltac=1.686
H0=67.7
h=H0/100
c=2.99*10**5
#Set up integration options
nlim=10000


def D(z):
    return ccl.growth_factor(cosmo,1/(1+z))*0.708

def f(z):
    return ccl.growth_rate(cosmo,1/(1+z))

def T(k):
    #bbks approx
    
    k=k*h
    q=k/(Omega_m*h**2*math.exp(-Omega_b*(1+(2*h)**0.5/Omega_m)))
    return math.log(1+2.34*q)/2.34/q*(1+3.89*q+(16.2*q)**2+(5.47*q)**3+(6.71*q)**4)**(-0.25)#ccl.get_transfer(math.log(k),a)
    
def Pm(k,z1):
    return ccl.linear_matter_power(cosmo, k*h, 1/(1+z1))*h**3

def Pz0(k):
     return ccl.linear_matter_power(cosmo, k*h, 1/(1+0))*h**3
 
def sigma_8(z1):
     return (Pm(0.01,z1)/Pz0(0.01))**0.5*ccl.sigma8(cosmo)
 
def sigma_chi(z):
    sigma_z=0.01
    return c*(1+z)/(ccl.h_over_h0(cosmo,1/(1+z))*h*100)*sigma_z/(1+z)
        
        
def Dl(z):
    return ccl.background.luminosity_distance(cosmo, 1/(1+z))

 
def bias(z1,gal):
    if gal=='LRG':
        return 1.7*D(0)/D(z1)#LRG
    if gal=='ELG':
        return D(0)/D(z1)#ELG
    if gal=='ELG_DESI':
        return 0.84*D(0)/D(z1)#ELG
    if gal=='ELG_MSE':
        return 0.8*D(0)/D(z1)#ELG
    if gal=='LBG24':
        return (-0.98*(24-25)+0.11)*(1+z1)+(0.12*(24-25)+0.17)*(1+z1)**2#LBG
    if gal=='LBG24.2':
        return (-0.98*(24.2-25)+0.11)*(1+z1)+(0.12*(24.2-25)+0.17)*(1+z1)**2#LBG
    if gal=='LBG24.5':
        return (-0.98*(24.5-25)+0.11)*(1+z1)+(0.12*(24.5-25)+0.17)*(1+z1)**2#LBG
    if gal=='LBG24.8':
        return (-0.98*(24.8-25)+0.11)*(1+z1)+(0.12*(24.8-25)+0.17)*(1+z1)**2
    if gal=='LBG25':
        return 0.11*(1+z1)+0.17*(1+z1)**2#LBG
    if gal=='LBG25.5':
        return (-0.98*(25.5-25)+0.11)*(1+z1)+(0.12*(25.5-25)+0.17)*(1+z1)**2#LBG
    if gal=='QSO':
        return 0.53+0.29*(1+z1)**2
    print('galaxy type unknown')
    
    
    
    
def Fnl_2param(z2,zeff,dz,n,sky,gal,kmax,m_max):
    Vsur=Vsurvey(z2,dz,sky)
    p=1
    kmin=2*math.pi/Vsur**(1/3)
    
    #fiducial
    z=zeff
    fNL=0
    if gal=='LBG':
        bg=(-0.98*(m_max-25)+0.11)*(1+zeff)+(0.12*(m_max-25)+0.17)*(1+zeff)**2
    else: 
        bg=bias(z,gal)
    
    def deltab(k):
        return 3*fNL*(bg-p)*deltac*Omega_m/(k**2*T(k)*D(z))*(H0/c)**2

    def P(k,z1,mu):
        return (bg+deltab(k)+f(z1)*mu*mu)**2*Pm(k,z1)

    def ddb_df(k):
        return 3*(bg-p)*deltac*Omega_m/(k**2*T(k)*D(z))*(H0/c)**2
    
    def ddb_db(k):#derivative of deltab by bg
        return 3*fNL*deltac*Omega_m/(k**2*T(k)*D(z))*(H0/c)**2
    
    def Fnl_bias():
        def Fij_fnl_bg(k,mu,int2):
            pk=P(k,z,mu)
            ddbdf=ddb_df(k)
            ddbdb=ddb_db(k)
            if int2==11:
                return 2*(n*pk/(n*pk+1))**2*ddbdf**2*k**2/(bg+deltab(k)+f(z)*mu*mu)**2
            if int2==12:
                return 2*(n*pk/(n*pk+1))**2*ddbdf*(1+ddbdb)*k**2/(bg+deltab(k)+f(z)*mu*mu)**2
            if int2==22:
                return 2*(n*pk/(n*pk+1))**2*(1+ddbdb)**2*k**2/(bg+deltab(k)+f(z)*mu*mu)**2
        
        def integrat_Fk_fnl_bg(muint,int1):
            if int1 ==11:
                return integrate.quad(partial(Fij_fnl_bg,mu=muint,int2=int1),kmin,kmax,epsrel=0.0001,epsabs=0.0001,limit=nlim)[0]
            if int1 ==12:
                return integrate.quad(partial(Fij_fnl_bg,mu=muint,int2=int1),kmin,kmax,epsrel=0.0001,epsabs=0.0001,limit=nlim)[0]
            if int1 ==22:
                return integrate.quad(partial(Fij_fnl_bg,mu=muint,int2=int1),kmin,kmax,epsrel=0.0001,epsabs=0.0001,limit=nlim)[0]
        def F_integrat_fnl_bg():
            
            f11=Vsur/(4*math.pi**2)*quad(partial(integrat_Fk_fnl_bg,int1=11),-1, 1,epsrel=0.0001,epsabs=0.001,limit=nlim)[0]
            f12=Vsur/(4*math.pi**2)*quad(partial(integrat_Fk_fnl_bg,int1=12),-1, 1,epsrel=0.0001,epsabs=0.0001,limit=nlim)[0]
            f22=Vsur/(4*math.pi**2)*quad(partial(integrat_Fk_fnl_bg,int1=22),-1, 1,epsrel=0.0001,epsabs=0.0001,limit=nlim)[0]
            return np.array([[f11,f12],[f12,f22]])
        
        def cov():
            return np.linalg.inv(F_integrat_fnl_bg())
        
        [[a1,b],[c1,d]]=cov()
        #print('The covariance matrix is', [[a1,b],[c1,d]])
        #print('sigma on fnl and bg are ', a1**0.5,' and ',d**0.5)
        return F_integrat_fnl_bg()

    return Fnl_bias()


kmax=0.1
print(' Megamapper kmax=0.1')
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00057,14000,'LBG24.5',kmax,24.5)+Fnl_2param(3,3.3,1,0.00011,14000,'LBG24.5',kmax,24.5)+Fnl_2param(4,4.3,1,0.00007,14000,'LBG24.5',kmax,24.5))[0][0]**0.5)
kmax=0.3
print(' Megamapper kmax=0.3')
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00057,14000,'LBG24.5',kmax,24.5)+Fnl_2param(3,3.3,1,0.00011,14000,'LBG24.5',kmax,24.5)+Fnl_2param(4,4.3,1,0.00007,14000,'LBG24.5',kmax,24.5))[0][0]**0.5)



kmax=0.1
print('\n','NTL for 1 tracer kmax=0.1')
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00065,14000,'LBG24.2',kmax,24.2)+Fnl_2param(3,3.3,1,0.00011,14000,'LBG24.2',kmax,24.2)+Fnl_2param(4,4.3,1,0.000024,14000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00012,14000,'LBG',kmax,24.5)+Fnl_2param(3,3.3,1,0.00027,14000,'LBG',kmax,24.5)+Fnl_2param(4,4.3,1,0.000075,14000,'LBG',kmax,24.5))[0][0]**0.5)
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.0029,14000,'LBG',kmax,25)+Fnl_2param(3,3.3,1,0.00088,14000,'LBG',kmax,25)+Fnl_2param(4,4.3,1,0.00033,14000,'LBG',kmax,25))[0][0]**0.5)
kmax=0.3
print('NTL for 1 tracer kmax=0.3')
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00065,14000,'LBG24.2',kmax,24.2)+Fnl_2param(3,3.3,1,0.00011,14000,'LBG24.2',kmax,24.2)+Fnl_2param(4,4.3,1,0.000024,14000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.00012,14000,'LBG',kmax,24.5)+Fnl_2param(3,3.3,1,0.00027,14000,'LBG',kmax,24.5)+Fnl_2param(4,4.3,1,0.000075,14000,'LBG',kmax,24.5))[0][0]**0.5)
print(np.linalg.inv(Fnl_2param(2,2.3,1,0.0029,14000,'LBG',kmax,25)+Fnl_2param(3,3.3,1,0.00088,14000,'LBG',kmax,25)+Fnl_2param(4,4.3,1,0.00033,14000,'LBG',kmax,25))[0][0]**0.5)

kmax=0.1
print('\n','MSE ELG kmax=0.1')
print(np.linalg.inv(Fnl_2param(1.6,2,0.8,0.00018,10000,'ELG_MSE',kmax,0))[0][0]**0.5)#lrg
kmax=0.3
print('MSE ELG kmax=0.3')
print(np.linalg.inv(Fnl_2param(1.6,2,0.8,0.00018,10000,'ELG_MSE',kmax,0))[0][0]**0.5)
kmax=0.1
print('MSE LBG kmax=0.1')
print(np.linalg.inv(Fnl_2param(2.4,2.55,0.8,0.00017,10000,'LBG24.2',kmax,24.2)+Fnl_2param(3.2,3.3,0.8,0.000043,10000,'LBG24.2',kmax,24.2))[0][0]**0.5)
kmax=0.3
print('MSE LBG kmax=0.1')
print(np.linalg.inv(Fnl_2param(2.4,2.55,0.8,0.00017,10000,'LBG24.2',kmax,24.2)+Fnl_2param(3.2,3.3,0.8,0.000043,10000,'LBG24.2',kmax,24.2))[0][0]**0.5)


kmax=0.1
print('\n','MUST kmax=0.1')
print('20k')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00146,15000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00044,15000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000044,15000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print('10k')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00085,15000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00026,15000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000026,15000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print('10k; 9kdeg2')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00085*1.68,9000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00026*1.68,9000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000026*1.68,9000,'LBG24.2',kmax,24.2))[0][0]**0.5)
kmax=0.3
print('MUST kmax=0.3')
print('20k')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00146,15000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00044,15000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000044,15000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print('10k')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00085,15000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00026,15000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000026,15000,'LBG24.2',kmax,24.2))[0][0]**0.5)
print('10k; 9kdeg2')
print(np.linalg.inv(Fnl_2param(2,2.2,0.8,0.00085*1.68,9000,'LBG24.2',kmax,24.2)+Fnl_2param(2.8,2.9,0.8,0.00026*1.68,9000,'LBG24.2',kmax,24.2)+Fnl_2param(3.6,3.7,0.6,0.000026*1.68,9000,'LBG24.2',kmax,24.2))[0][0]**0.5)


 Megamapper kmax=0.1
1.3208564340500273
 Megamapper kmax=0.3
1.1722451737609743

 NTL for 1 tracer kmax=0.1
1.412511198683495
1.3068693406761083
1.146036662169775
NTL for 1 tracer kmax=0.3
1.259481817326091
1.185472794389647
0.9511813438372005

 MSE ELG kmax=0.1
8.882352602258965
MSE ELG kmax=0.3
8.095257121902776
MSE LBG kmax=0.1
2.7592961000472727
MSE LBG kmax=0.1
2.4832838783576623

 MUST kmax=0.1
20k
1.5596259844039015
10k
1.679443923791133
10k; 9kdeg2
2.1392930410636075
MUST kmax=0.3
20k
1.3653855007011826
10k
1.4831508166740484
10k; 9kdeg2
1.8467593459240197


# Now for two tracers

In [9]:
def fnl_2tracer(z2,zeff,dz,na,nb,sky,gal1,gal2,kmax):

    z=zeff#z2#+dz/2
    sigma8=sigma_8(z)
    sigma8_0=sigma_8(0)
    f_z=f(z)
    ba=bias(z,gal1)
    bb=bias(z,gal2)
    V=Vsurvey(z2,dz,sky)
    p=1
    kmin=2*math.pi/V**(1/3)
    
    def PA(k,mu):
        return (ba+f_z*mu**2)**2*Pm(k,z)*math.exp(-k**2*mu**2*sigma_chi(z)**2)
    def PB(k,mu):
        return (bb+f_z*mu**2)**2*Pm(k,z)*math.exp(-k**2*mu**2*sigma_chi(z)**2)
    def PAB(k,mu):
        return (ba+f_z*mu**2)*(bb+f_z*mu**2)*Pm(k,z)*math.exp(-k**2*mu**2*sigma_chi(z)**2)
    def ddb_df(k,bg):
        return 3*(bg-p)*deltac*Omega_m/(k**2*T(k)*D(z))*(H0/c)**2
    
    
    def Dt(X,i,mu,k):
        if X=='A':
            if i==1:
                return 2/(ba+f_z*mu**2)
            if i==2:
                return 0
            if i==3:
                return 2*ddb_df(k,ba)/(ba+f_z*mu**2)
        if X=='B':
            if i==1:
                return 0
            if i==2:
                return 2/(bb+f_z*mu**2)
            if i==3:
                return 2*ddb_df(k,bb)/(bb+f_z*mu**2)
        if X=='AB':
            if i==1:
                return 1/(ba+f_z*mu**2)
            if i==2:
                return 1/(bb+f_z*mu**2)
            if i==3:
                return ddb_df(k,bb)/(bb+f_z*mu**2)+ddb_df(k,ba)/(ba+f_z*mu**2)
        print('error DX')
    
    def Raa(k,mu):
        return (na*PA(k,mu)*(1+nb*PB(k,mu))/((1+na*PA(k,mu))*(1+nb*PB(k,mu))-na*nb*PAB(k,mu)**2))**2
    def Rbb(k,mu):
        return (nb*PB(k,mu)*(1+na*PA(k,mu))/((1+nb*PB(k,mu))*(1+na*PA(k,mu))-na*nb*PAB(k,mu)**2))**2
    def Rxx(k,mu):
        return na*nb*((1+na*PA(k,mu))*(1+nb*PB(k,mu))+na*nb*PAB(k,mu)**2)/((1+na*PA(k,mu))*(1+nb*PB(k,mu))-na*nb*PAB(k,mu)**2)**2*PAB(k,mu)**2
    
    def Rxa(k,mu):
        return na**2*nb*(1+nb*PB(k,mu))/((1+na*PA(k,mu))*(1+nb*PB(k,mu))-na*nb*PAB(k,mu)**2)**2*PAB(k,mu)**2*PA(k,mu)
    def Rxb(k,mu):
        return nb**2*na*(1+na*PA(k,mu))/((1+na*PA(k,mu))*(1+nb*PB(k,mu))-na*nb*PAB(k,mu)**2)**2*PAB(k,mu)**2*PB(k,mu)
    def Rab(k,mu):
        return na**2*nb**2*PA(k,mu)*PB(k,mu)*PAB(k,mu)**2/((1+na*PA(k,mu))*(1+nb*PB(k,mu))-na*nb*PAB(k,mu)**2)**2
    
    
    def FX(X,k,mu,i,j):
        if X=='A':
            return 1/2*Dt(X,i,mu,k)*Dt(X,j,mu,k)*Raa(k,mu)
        if X=='B':
            return 1/2*Dt(X,i,mu,k)*Dt(X,j,mu,k)*Rbb(k,mu)
        if X=='AB':
            return Dt(X,i,mu,k)*Dt(X,j,mu,k)*Rxx(k,mu)-(Dt(X,i,mu,k)*Dt('A',j,mu,k)+Dt('A',i,mu,k)*Dt(X,j,mu,k))*Rxa(k,mu)-(Dt(X,i,mu,k)*Dt('B',j,mu,k)+Dt('B',i,mu,k)*Dt(X,j,mu,k))*Rxb(k,mu)+1/2*(Dt('A',i,mu,k)*Dt('B',j,mu,k)+Dt('B',i,mu,k)*Dt('A',j,mu,k))*Rab(k,mu)
        print('error Fx')
    def integrand(k,mu,i,j):
        return (FX('A',k,mu,i,j)+FX('B',k,mu,i,j)+FX('AB',k,mu,i,j))*k**2
    
        
    def integrat_rsd(muint,int_i,int_j):
        return integrate.quad(partial(integrand,mu=muint,i=int_i,j=int_j),kmin,kmax,epsrel=0.0000001,epsabs=0,limit=nlim)[0]
    
    def F_rsd():
        #print('ok')
        f11=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=1,int_j=1),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        f12=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=1,int_j=2),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        f13=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=1,int_j=3),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        f22=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=2,int_j=2),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        f23=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=2,int_j=3),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        f33=V/(4*math.pi**2)*quad(partial(integrat_rsd,int_i=3,int_j=3),-1, 1,epsrel=0.001,epsabs=0.0,limit=nlim)[0]
        return np.array([[f11,f12,f13],[f12,f22,f23],[f13,f23,f33]])
    Ff=F_rsd()
    return Ff



#%%MUST
print('MUST 2 tracers kmax=0.1')
kmax=0.1
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.0007,0.00014,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00021,0.000042,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000021,0.0000042,15000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.00035,0.00007,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00011,0.000021,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000011,0.0000021,15000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.00035*1.68,0.00007*1.68,9000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00011*1.68,0.000021*1.68,9000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000021*1.68,0.0000042*1.68,9000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)

print('MUST 2 tracers kmax=0.3')
kmax=0.3
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.0007,0.00014,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00021,0.000042,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000021,0.0000042,15000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.00035,0.00007,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00011,0.000021,15000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000011,0.0000021,15000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.2,0.8,0.00035*1.68,0.00007*1.68,9000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(2.8,2.9,0.8,0.00011*1.68,0.000021*1.68,9000,'LBG24.2','LBG25.5',kmax)+fnl_2tracer(3.6,3.7,0.6,0.000021*1.68,0.0000042*1.68,9000,'LBG24.2','LBG25.5',kmax))[2][2]**0.5)

#%%
print('\n','NTL 2 tracers kmax=0.1')
kmax=0.1
print(np.linalg.inv(fnl_2tracer(2,2.3,1,0.00065,0.00055,15000,'LBG24.2','LBG24.5',kmax)+fnl_2tracer(3,3.3,1,0.00011,0.00016,15000,'LBG24.2','LBG24.5',kmax)+fnl_2tracer(4,4.3,1,0.000024,0.00005,15000,'LBG24.2','LBG24.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.3,1,0.0012,0.0017,15000,'LBG24.5','LBG25',kmax)+fnl_2tracer(3,3.3,1,0.00027,0.00061,15000,'LBG24.5','LBG25',kmax)+fnl_2tracer(4,4.3,1,0.000075,0.00025,15000,'LBG24.5','LBG25',kmax))[2][2]**0.5)
print('NTL 2 tracers kmax=0.3')
kmax=0.3
print(np.linalg.inv(fnl_2tracer(2,2.3,1,0.00065,0.00055,15000,'LBG24.2','LBG24.5',kmax)+fnl_2tracer(3,3.3,1,0.00011,0.00016,15000,'LBG24.2','LBG24.5',kmax)+fnl_2tracer(4,4.3,1,0.000024,0.00005,15000,'LBG24.2','LBG24.5',kmax))[2][2]**0.5)
print(np.linalg.inv(fnl_2tracer(2,2.3,1,0.0012,0.0017,15000,'LBG24.5','LBG25',kmax)+fnl_2tracer(3,3.3,1,0.00027,0.00061,15000,'LBG24.5','LBG25',kmax)+fnl_2tracer(4,4.3,1,0.000075,0.00025,15000,'LBG24.5','LBG25',kmax))[2][2]**0.5)


MUST 2 tracers kmax=0.1
1.6797983635653986
1.9326836877032891
2.2271604166725534
MUST 2 tracers kmax=0.3
1.538059205612595
1.7871132443547912
2.024625218348287

 NTL 2 tracers kmax=0.1
1.1432625928959688
1.0284181656467164
NTL 2 tracers kmax=0.3
1.035053899466604
0.9021200465486707
